In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime as dt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from pylab import plt
plt.style.use('seaborn')
import matplotlib as mpl
mpl.rcParams['font.family'] = 'serif'

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import pandas as pd
import cufflinks as cf

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [ ]:
# data from FXCM Forex Capital Markets Ltd.
raw = pd.read_csv('../input/fxcm_eur_usd_eod_data.csv',
                 index_col=0, parse_dates=True)
raw.columns

In [ ]:
quotes = raw[['OpenAsk', 'HighAsk', 'LowAsk', 'CloseAsk']]

In [ ]:
qf = cf.QuantFig(quotes.iloc[-100:], title='EUR/USD', legend='top',
                 name='EUR/USD', datalegend=False)

In [ ]:
'''from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

iplot(qf.iplot(asFigure=True))
qf.add_bollinger_bands(periods=15, boll_std=2)
iplot(qf.iplot(asFigure=True))'''

In [ ]:
# data from Thomson Reuters Eikon API
raw = pd.read_csv('../input/tr_eikon_eod_data.csv',
                 index_col=0, parse_dates=True)
raw.info()

In [ ]:
data = pd.DataFrame(raw['.SPX'])
data.columns = ['Close']

In [ ]:
data.tail()

In [ ]:
data['Close'].plot(figsize=(8, 5), grid=True);


In [ ]:
%time data['Return'] = np.log(data['Close'] / data['Close'].shift(1))

In [ ]:
data[['Close', 'Return', 'Return']].tail()

In [ ]:
data[['Close', 'Return']].plot(subplots=True, style='b',
                              figsize=(8, 5), grid=True);

In [ ]:
data['42d'] = data['Close'].rolling(window=42).mean()
data['252d'] = data['Close'].rolling(window=252).mean()
data[['Close', '42d', '252d']].tail()
data[['Close', '42d', '252d']].plot(figsize=(8, 5), grid=True)

In [ ]:
import math
data['Mov_Vol'] = data['Return'].rolling(window=252).std() * math.sqrt(252)
  # moving annual volatility

In [ ]:
data[['Close', 'Mov_Vol', 'Return']].plot(subplots=True, style='b',
                                         figsize=(8, 7), grid=True);

****Regression Analysis****

In [ ]:
# data from Thomson Reuters Eikon API
raw = pd.read_csv('../input/tr_eikon_eod_data.csv',
                 index_col=0, parse_dates=True)
spx = pd.DataFrame(raw['.SPX'])

In [ ]:
np.round(spx.tail())

In [ ]:
vix = pd.DataFrame(raw['.VIX'])
vix.info()

In [ ]:
data = spx.join(vix)

In [ ]:
data.tail()

In [ ]:
data.plot(subplots=True, grid=True, style='b', figsize=(8, 6));
# tag: spx_vix
# title: The S&P 500 Index and the VIX volatility index

In [ ]:
rets = np.log(data / data.shift(1)) 
rets.head()

In [ ]:
rets.dropna(inplace=True)

In [ ]:
rets.plot(subplots=True, grid=True, style='b', figsize=(8, 6));
# tag: es50_vs_rets
# title: Log returns of S&P500 and VIX

In [ ]:
xdat = rets['.SPX'].values
ydat = rets['.VIX'].values
reg = np.polyfit(x=xdat, y=ydat, deg=1)
reg

In [ ]:
plt.plot(xdat, ydat, 'r.')
ax = plt.axis()  # grab axis values
x = np.linspace(ax[0], ax[1] + 0.01)
plt.plot(x, np.polyval(reg, x), 'b', lw=2)
plt.grid(True)
plt.axis('tight')
plt.xlabel('S&P 500 returns')
plt.ylabel('VIX returns')
# tag: scatter_rets
# title: Scatter plot of log returns and regression line

In [ ]:
rets.corr()

In [ ]:
rets['.SPX'].rolling(window=252).corr(rets['.VIX']).plot(grid=True, style='b')
# tag: roll_corr
# title: Rolling correlation between S&P 500 and VIX

In [ ]:
# data from FXCM Forex Capital Markets Ltd.
eur_usd = pd.read_csv('../input/fxcm_eur_usd_tick_data.csv',
                     index_col=0, parse_dates=True)

In [ ]:
eur_usd.info()

In [ ]:
eur_usd['Mid'] = eur_usd.mean(axis=1)

In [ ]:
eur_usd['Mid'].plot(figsize=(10, 6));
# tag: eur_uusd
# title: EUR/USD tick data for two hours

In [ ]:
eur_usd_resam = eur_usd.resample(rule='1min', label='left').last()
eur_usd_resam.head()

In [ ]:
eur_usd_resam['Mid'].plot(grid=True)
# tag: eur_usd_resam
# title: Resampled EUR/USD exchange rate tick data

In [ ]:
def reversal(x):
    return 2 * 1.16 - x

In [ ]:
eur_usd_resam['Mid'].apply(reversal).plot()
# tag: aapl_resam_apply
# title: Resampled EUR/USD exchange rate tick data with function applied to it